In [102]:
#必要ライブラリの読み込み
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, models
from torch.utils.data import DataLoader
from torch.optim import Adam
import os
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import joblib
from PIL import Image
import torchvision.transforms as T
from torch.utils.data import Dataset
import torch.optim as optim

print('hello')

hello


In [104]:
# データの読み込みと編集
data = pd.read_csv('banana_brix9.csv')  # CSVファイルに画像ファイル名とクラスラベルが含まれていると仮定
# クラスラベルを整数にエンコードする辞書を作成
label_encoding = {
    'non-ripe': 0,
    'sweet': 1,
    'rotten': 2
}

# データフレームの'judge'列を整数にエンコードして新しい'label'列を作成
data['label'] = data['judge'].map(label_encoding)

# 'label'列が正しく追加されたことを確認
data

,id,image,brix,check,judge,label
0,1.0,bbrix1.JPG,17.0,NaN,non-ripe,0
1,2.0,bbrix2-1.JPG,20.0,NaN,rotten,2
2,4.0,bbrix3.JPG,22.0,NaN,sweet,1
3,5.0,bbrix4-1.JPG,17.0,NaN,non-ripe,0
4,6.0,bbrix4-2.JPG,17.0,NaN,non-ripe,0
...,...,...,...,...,...,...
122,NaN,n111.JPG,NaN,NaN,sweet,1
123,NaN,n112.JPG,NaN,NaN,sweet,1
124,NaN,n113.JPG,NaN,NaN,sweet,1
125,NaN,n114.JPG,NaN,NaN,sweet,1


In [105]:
#バナナ検知モデルを使い、編集画像を保存する
# モデルを読み込む
model_d = joblib.load('bananaDetect3.pkl')
# 画像フォルダのパス
image_folder = 'banana_brix_images'
edit_image_folder = 'banana_brix_images_edit'
# デバイスの指定 (GPUが利用可能な場合はGPUを使用し、それ以外の場合はCPUを使用)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



# 1. 画像の前処理
def preprocess_image(image):
    transform = T.Compose([
        T.ToTensor(),  # 画像をテンソルに変換
    ])
    return transform(image).unsqueeze(0)  # バッチ次元を追加

# 2. モデルに画像を渡して推論
def predict(model, image_tensor):
    model.eval()
    with torch.no_grad():
        image_tensor = image_tensor.to(device)
        predictions = model(image_tensor)

    return predictions

# 画像フォルダ内の各画像に対して処理
for filename in os.listdir(image_folder):
    if filename.endswith('.JPG'):
        image_path = os.path.join(image_folder, filename)
        # 画像を開く
        pil_image = Image.open(image_path)
        # バウンディングボックスを検出
        image_tensor = preprocess_image(pil_image)
        predictions = predict(model_d, image_tensor)
        
        # しきい値を設定
        threshold = 0.5  # この値を必要に応じて調整
        # 最もスコアが高いバウンディングボックスを見つける
        best_score = 0.0
        best_box = None
        for score, label, box in zip(predictions[0]['scores'], predictions[0]['labels'], predictions[0]['boxes']):
            if score > threshold and score > best_score:
                best_score = score
                best_box = box

        if best_box is not None:
            best_box = [round(i, 2) for i in best_box.tolist()]
            x1, y1, x2, y2 = map(int, best_box)
            cropped_image = pil_image.crop((x1, y1, x2, y2))
            # 画像を保存
            edit_image_path = os.path.join(edit_image_folder, filename)
            cropped_image.save(edit_image_path)

print("編集済みの画像を保存しました。")

編集済みの画像を保存しました。


In [106]:
#編集画像のパスと前処理
# 画像フォルダのパス
image_folder = 'banana_brix_images_edit'  # 保存した編集済み画像のフォルダ

# 画像の前処理
transform = T.Compose([
    T.Resize((224, 224)),  # 画像を指定のサイズにリサイズ
    T.RandomHorizontalFlip(),  # ランダムな水平反転
    T.RandomRotation(10),  # ランダムな回転（最大10度）
    T.ToTensor(),  # 画像をテンソルに変換
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 画像を標準化
])

In [107]:
# 除外するファイル名のリスト
exclude_filenames = ['n20.JPG']
# 'image' 列が特定のファイル名を持たない行を取り除く
data = data[~data['image'].isin(exclude_filenames)]


In [108]:
#データセットの設定
class CustomDataset(Dataset):
    def __init__(self, dataframe, image_folder, transform=None):
        self.dataframe = dataframe
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, str(self.dataframe.iloc[idx]['image']))  # 'image'列を文字列に変換
        # ファイルの存在確認
        if os.path.exists(img_name):
            image = Image.open(img_name)
            label = int(self.dataframe.iloc[idx]['label'])  # 'label'列を使用
        else:
            # ファイルが存在しない場合の処理を記述
            print(f"File not found: {img_name}")
            # 例外をスローする場合： raise FileNotFoundError(f"File not found: {img_name}")
            # デフォルト画像を使用する場合： image = Image.open('default_image.jpg')
            return None, None  # ダミーの値を返すことでスキップします

        if self.transform:
            image = self.transform(image)

        return image, label

In [109]:
import torch.nn.init as init

#分割しないでみる
dataset=CustomDataset(data,image_folder,transform=transform)
batch_size = 15
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


# ニューラルネットワークモデルの定義 (例: ResNet)
model = models.resnet18(pretrained=True)
num_classes = 3  # クラス数を指定

# モデルの最終層を変更して、出力クラス数に合わせる
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Heの初期化を適用
def initialize_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        init.kaiming_normal_(m.weight)
        if m.bias is not None:
            init.zeros_(m.bias)

# モデルの各層の重みを初期化
print('before')
model.apply(initialize_weights)
print('after')

# 損失関数とオプティマイザを定義
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# モデルの訓練
num_epochs = 6  # エポック数を指定 (必要に応じて調整)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # エポックごとの損失を表示
    print(f'Epoch [{epoch + 1}/{num_epochs}] Loss: {running_loss / len(loader)}')

print('Finished Training')


/Users/taisei_yamaguchi/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/taisei_yamaguchi/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


before
after
Epoch [1/6] Loss: 5.615104966693455
Epoch [2/6] Loss: 2.289085202746921
Epoch [3/6] Loss: 1.7477173739009433
Epoch [4/6] Loss: 1.2803681426578097
Epoch [5/6] Loss: 1.2320668896039326
Epoch [6/6] Loss: 1.0324179066552057
Finished Training


In [110]:

# 1. モデルを読み込む
model_d = joblib.load('bananaDetect3.pkl')

# 2. 画像の前処理
def preprocess_image(image):
    transform_d = T.Compose([
        T.ToTensor(),  # 画像をテンソルに変換
    ])
    return transform_d(image).unsqueeze(0)  # バッチ次元を追加

# 3. モデルに画像を渡してバウンディングボックスを検出
def detect_bounding_box(model, image_tensor, threshold=0.5):
    model.eval()
    with torch.no_grad():
        image_tensor = image_tensor.to(device)
        predictions = model(image_tensor)

    # しきい値を超える最もスコアの高いバウンディングボックスを見つける
    best_score = 0.0
    best_box = None
    for score, label, box in zip(predictions[0]['scores'], predictions[0]['labels'], predictions[0]['boxes']):
        if score > threshold and score > best_score:
            best_score = score
            best_box = box

    return best_box



#4. 切り取った画像に対する前処理を追加
def preprocess_cropped_image(image):
    transform = T.Compose([
        T.Resize((224, 224)),  # 画像を指定のサイズにリサイズ
        T.RandomHorizontalFlip(),  # ランダムな水平反転
        T.RandomRotation(10),  # ランダムな回転（最大10度）
        T.ToTensor(),  # 画像をテンソルに変換
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 画像を標準化
    ])
    return transform(image).unsqueeze(0)  # バッチ次元を追加

# 5. 切り取った画像に対して分類を行う
def classify_banana(model, image):
    # 画像の前処理を適用
    image_tensor = preprocess_image(image)
    
    #ここで、切り取り画像の前処理を行う。
    
    # モデルに画像を入力し、予測を取得
    with torch.no_grad():
        outputs = model(image_tensor)
    
    # 予測クラスの取得
    _, predicted_class = outputs.max(1)
    
    return predicted_class.item()




# 画像フォルダ内の各画像に対して処理
image_folder = 'bananas_images_test'  # 画像フォルダのパスを指定

for filename in os.listdir(image_folder):
    if filename.endswith('.JPG'):
        image_path = os.path.join(image_folder, filename)
        # 画像を開く
        pil_image = Image.open(image_path)
        # バウンディングボックスを検出
        image_tensor = preprocess_image(pil_image)
        bounding_box = detect_bounding_box(model_d, image_tensor)
        
        if bounding_box is not None:
            # バウンディングボックス情報を取得
            x1, y1, x2, y2 = map(int, bounding_box.tolist())
            cropped_image = pil_image.crop((x1, y1, x2, y2))
            
            # 切り取り画像に前処理を適用
            cropped_image_tensor = preprocess_cropped_image(cropped_image)
            
            
            # 分類を実行
            predicted_class = classify_banana(model, cropped_image)
            
            # 予測クラスを表示
            print(f"Predicted class for {filename}: {predicted_class}")
            
            
print('finish')

Predicted class for b53.JPG: 1
Predicted class for b47.JPG: 1
Predicted class for bbrix28.JPG: 1
Predicted class for b52.JPG: 1
Predicted class for b44.JPG: 1
Predicted class for n48.JPG: 1
Predicted class for b40.JPG: 1
Predicted class for b43.JPG: 1
Predicted class for bbrix4-1.JPG: 1
Predicted class for bbrix11-1.JPG: 1
Predicted class for n38.JPG: 1
Predicted class for n39.JPG: 1
Predicted class for b39.JPG: 1
Predicted class for n35.JPG: 1
Predicted class for n37.JPG: 1
Predicted class for n36.JPG: 1
Predicted class for n43.JPG: 1
Predicted class for n42.JPG: 1
Predicted class for b4.JPG: 1
Predicted class for n40.JPG: 1
Predicted class for n41.JPG: 1
Predicted class for b3.JPG: 1
Predicted class for n51.JPG: 1
Predicted class for bbrix18-2.JPG: 1
Predicted class for n50.JPG: 1
Predicted class for n44.JPG: 1
Predicted class for b49.JPG: 1
Predicted class for b2.JPG: 1
Predicted class for bbrix18-1.JPG: 1
Predicted class for n52.JPG: 1
Predicted class for b1.JPG: 1
Predicted class 

# 真っ黒いバナナの画像を除いて学習？